In [1]:
import pandas as pd
import json
import os
import ast

In [2]:
LOCATIE_REQUESTS = os.path.join("../data/requests")
RUWE_DATA_CSV = os.path.join('../data/ruw/all_ruwe_data.csv')
GEKREGEN_EXCEL_FILE = os.path.join('../data/ModifiedQueryRows.xlsx')

In [3]:
def parse_filename(file_name) -> tuple:
    try:
        # rsplit verwijdert alleen de laatste .json extensie (voor mocht het bestand meerdere .json bevatten alhoewel dit onwaarschijnlijk lijkt)
        parts = file_name.rsplit('.json', 1)[0].split('-')
        if len(parts) >= 5:
            route_id = parts[0]
            date = parts[1]
            time = parts[2]
            number_of_tasks = parts[3]
            number_of_tasks_in_input_plan = parts[4]
            return route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan
        else:
            print(f"Skipping file {file_name}: incorrect format")
            return None
    except Exception as e:
        print(f"Error parsing file name {file_name}: {e}")
        return None

In [4]:
# check if csv bestand bestaat
def check_csv_bestand(bestandspadennaam) -> bool:
    if os.path.exists(bestandspadennaam):
        return True
    return False

# lees csv bestand in als dataframe
def lees_dataframe_uit_csv(bestandspadennaam) -> pd.DataFrame:
    df = pd.read_csv(bestandspadennaam)  
    return df

# maak csv bestand aan vanuit dataframe
def schrijf_dataframe_naar_csv(df: pd.DataFrame, bestandspadennaam) -> None:
    # Maak de directory aan als deze niet bestaat
    os.makedirs(os.path.dirname(bestandspadennaam), exist_ok=True)
    # Schrijf het dataframe naar CSV
    df.to_csv(bestandspadennaam, index=False)
    return None

# de verschillende json-bestanden van de requests inladen en samenvoegen tot 1 grote dataframe 
# geeft een dataframe terug als resultaat
def lees_json_bestanden_en_maak_dataframe(locatie_requests) -> pd.DataFrame:
    df = pd.DataFrame()
    for folder_name in os.listdir(locatie_requests):
        folder_path = os.path.join(locatie_requests, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.json'):
                    parsed_data = parse_filename(file_name)
                    if parsed_data == None:
                        break
                    else:
                        route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan = parsed_data
                        # print(route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan)
                        file_path = os.path.join(folder_path, file_name)
                        with open(file_path, 'r') as f:
                            data = json.load(f)
                            # Voeg route_id toe aan elke record in data
                            data['route_id'] = route_id
                            data['date'] = date
                            data['time'] = time
                            data['number_of_tasks'] = number_of_tasks
                            data['number_of_tasks_in_input_plan'] = number_of_tasks_in_input_plan
                            temp_df = pd.DataFrame([data])
                            # voeg tijdelijke dataframe toe aan de hoofddataframe
                            df = pd.concat([df, temp_df], ignore_index=True)
    
    return df


In [17]:

if (check_csv_bestand(RUWE_DATA_CSV) == False):
    # maak het bestand een eerste keer

    ingelezen_dataframe = lees_json_bestanden_en_maak_dataframe(LOCATIE_REQUESTS)
    # dit is dan dezelfde dataframe als in de gegeven excel file maar met tasks en fixedTasks eraan toegevoegd en zonder TriggerType
    # date en time zijn nog steeds strings, kan later nog omgezet worden naar datetime indien nodig
    # timecalculation, 
    display(ingelezen_dataframe.head())

    schrijf_dataframe_naar_csv(ingelezen_dataframe, RUWE_DATA_CSV)

    display(ingelezen_dataframe.tail())
    print(f"Lengte van de dataframe: {len(ingelezen_dataframe)}")
    # excel inlezen om de lengte te checken
    df_excel = pd.read_excel(GEKREGEN_EXCEL_FILE)
    print(f"Aantal rijen in ingelezen excel dataframe: {len(df_excel)}")
    if (len(ingelezen_dataframe) != len(df_excel)):
        print("Waarschuwing: Aantal rijen in ingelezen dataframe komt niet overeen met aantal rijen in excel dataframe!")

else:
    # lees het bestand en voeg toe aan dataframe om verder mee te werken
    ingelezen_dataframe = lees_dataframe_uit_csv(RUWE_DATA_CSV)
    display(ingelezen_dataframe.tail())


,id,configurationName,tasks,fixedTasks,route_id,date,time,number_of_tasks,number_of_tasks_in_input_plan
21701,642acef2-3942-4057-aeb8-61df9f7ffa1b,CreateSequence,"[{'id': '63784', 'address': {'latitude': 0.482...",[],0521_O69,20220622,64101,135,0
21702,2d4fe424-cf93-49ea-afdc-6475b244fe69,CreateSequence,"[{'id': '63759', 'address': {'latitude': 0.487...",[],0521_O69,20220622,64811,139,0
21703,0e9bc064-e6ee-421b-ae3f-e09b39f6cf86,CreateSequence,"[{'id': '63652', 'address': {'latitude': 0.485...",[],0521_O69,20220622,72840,139,0
21704,c3b3eea2-dff2-4680-9002-b49ee714ca2d,EstimateTime,"[{'id': '63788', 'address': {'latitude': 0.487...","[{'taskId': '63788', 'activityType': 'Task', '...",0521_O69,20220622,73145,139,139
21705,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,EstimateTime,"[{'id': '63788', 'address': {'latitude': 0.487...","[{'taskId': '63788', 'activityType': 'Task', '...",0521_O69,20220622,83833,139,139


In [42]:
# dataframe met tasks maken
taken_df = ingelezen_dataframe[['id', 'tasks']].copy()
# type van tasks is een string dus moeten we dit eerst omzetten
def safe_eval(x):
    if pd.isna(x) or x == '':
        return []
    try:
        return ast.literal_eval(x)
    except Exception:
        return []


taken_df['tasks'] = taken_df['tasks'].apply(safe_eval)
taken_df = taken_df.explode('tasks')
tasks_normalized = pd.json_normalize(taken_df['tasks'])

tasks_normalized = tasks_normalized.rename(columns={
    'id': 'task_id',
    'address.latitude': 'latitude',
    'address.longitude': 'longitude',
    'timeWindow.from': 'from',
    'timeWindow.till': 'till'
})
tasks_normalized['id'] = taken_df['id'].values
tasks_normalized = tasks_normalized.reset_index(drop=True)

display(tasks_normalized.head())



,task_id,latitude,longitude,from,till,id
0,395,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,41931cd2-8975-4a64-9197-d16abe871bb7
1,394,0.565826,0.221868,2022-06-17T07:30:00,2022-06-17T18:00:00,41931cd2-8975-4a64-9197-d16abe871bb7
2,385,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,5737d4ef-23e9-4832-8bb5-24340a176e31
3,384,0.565826,0.221868,2022-06-17T07:30:00,2022-06-17T23:59:00,5737d4ef-23e9-4832-8bb5-24340a176e31
4,390,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,34f409b3-9f92-4720-9926-48ca1cbf6f90


In [43]:
# dataframe met fixedTasks maken
fixed_taken_df = ingelezen_dataframe[['id', 'fixedTasks']].copy()
# filter om alleen rijen te tonen waar fixedTasks niet leeg is
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'].astype(str) != '[]']

fixed_taken_df['fixedTasks'] = fixed_taken_df['fixedTasks'].apply(safe_eval)
# fixedTasks omzetten van string naar object
fixed_taken_df = fixed_taken_df.explode('fixedTasks')

# alleen rijen behouden waar fixedTasks niet leeg is
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'].notna()]
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'] != {}]

# normaliseren van fixedTasks
fixed_tasks_normalized = pd.json_normalize(fixed_taken_df['fixedTasks'])

# kolommen hernoemen voor consistentie
if not fixed_tasks_normalized.empty:
    # voeg id kolom toe
    fixed_tasks_normalized['id'] = fixed_taken_df['id'].values
    fixed_tasks_normalized = fixed_tasks_normalized.reset_index(drop=True)

display(fixed_tasks_normalized.head())

,taskId,activityType,fixedPosition,id
0,64869,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8
1,64947,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8
2,64971,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8
3,64878,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8
4,64926,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8


In [45]:
# Unieke locatie IDs toevoegen aan tasks_normalized
# Punten binnen 6 meter van elkaar krijgen dezelfde location_id

from sklearn.cluster import DBSCAN
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Bereken de afstand tussen twee punten op aarde in meters
    """
    # Aardstraal in meters
    R = 6371000
    
    # Converteer naar radialen
    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    
    # Haversine formule
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    
    return distance

# Bereid data voor clustering voor
coords = tasks_normalized[['latitude', 'longitude']].copy()

# Verwijder rijen met missende coördinaten
coords_clean = coords.dropna()

# Converteer naar radialen voor DBSCAN met haversine metric
coords_rad = np.radians(coords_clean[['latitude', 'longitude']].values)

# DBSCAN clustering
# eps in radialen: 6 meter / 6371000 meter (aardstraal) ≈ 0.000000941
eps_rad = 6 / 6371000  
dbscan = DBSCAN(eps=eps_rad, min_samples=1, metric='haversine')
clusters = dbscan.fit_predict(coords_rad)

# Voeg cluster labels toe aan de originele dataframe
tasks_normalized['location_id'] = np.nan
tasks_normalized.loc[coords_clean.index, 'location_id'] = clusters

# Converteer naar integer waar mogelijk
tasks_normalized['location_id'] = tasks_normalized['location_id'].astype('Int64')

# Toon resultaat
print(f"Aantal unieke locaties: {tasks_normalized['location_id'].nunique()}")
print(f"Aantal taken: {len(tasks_normalized)}")
display(tasks_normalized[['task_id', 'latitude', 'longitude', 'location_id']].head(20))

# Toon een voorbeeld van taken die dezelfde locatie delen
duplicated_locations = tasks_normalized[tasks_normalized.duplicated(subset=['location_id'], keep=False)]
if len(duplicated_locations) > 0:
    print(f"\nVoorbeeld van {len(duplicated_locations)} taken die locaties delen:")
    display(duplicated_locations.sort_values('location_id').head(10))

Aantal unieke locaties: 108621
Aantal taken: 2576770


,task_id,latitude,longitude,location_id
0,395,0.565826,0.221868,0
1,394,0.565826,0.221868,0
2,385,0.565826,0.221868,0
3,384,0.565826,0.221868,0
4,390,0.565826,0.221868,0
5,388,0.572716,0.233306,1
6,75,0.565826,0.221868,0
7,72,0.564949,0.222236,2
8,72,0.564949,0.222236,2
9,78,0.564949,0.222236,2



Voorbeeld van 2574395 taken die locaties delen:


,task_id,latitude,longitude,from,till,id,location_id
2396995,81576,0.565826,0.221868,2022-06-09T07:30:00,2022-06-09T23:59:00,692bb9c9-949e-4c87-b3a1-d2b3a50b9a53,0
2398192,82738,0.565826,0.221868,2022-06-16T07:30:00,2022-06-16T17:00:00,d4df82e7-7134-4337-821b-adfd9c7bfbce,0
2398189,82738,0.565826,0.221868,2022-06-16T07:30:00,2022-06-16T17:00:00,986b5b81-a575-4901-8190-4fceef35dcf0,0
2398239,82738,0.565826,0.221868,2022-06-16T07:30:00,2022-06-16T17:00:00,bac63966-f901-4313-96c6-f2167e1dc1d5,0
2397860,82452,0.565826,0.221868,2022-06-14T07:30:00,2022-06-14T17:00:00,548e4d1c-422b-43eb-93eb-f81ca66ac3f1,0
2401050,9679,0.565826,0.221868,2022-06-20T07:30:00,2022-06-20T23:59:00,7c1856dc-d1bb-406c-aa57-91d659a65f93,0
2401070,9753,0.565826,0.221868,2022-06-21T07:30:00,2022-06-21T23:59:00,7a71edb2-117d-40c5-a8e3-6c86e8f6f735,0
2397803,82452,0.565826,0.221868,2022-06-14T07:30:00,2022-06-14T17:00:00,3318980a-18e4-4f72-94ce-fe7f96ee3937,0
2401502,9753,0.565826,0.221868,2022-06-21T07:30:00,2022-06-21T23:59:00,e8e5692b-1c74-4b8b-84eb-aec28a1fda75,0
2397804,82452,0.565826,0.221868,2022-06-14T07:30:00,2022-06-14T17:00:00,8c603c55-574c-452c-ba70-eef7162adf87,0


In [46]:
# Voeg location_id toe aan fixed_tasks_normalized
# Match op basis van task_id (van fixed_tasks) en id (route id)

if not fixed_tasks_normalized.empty:
    # Check welke kolom in fixed_tasks_normalized de task id bevat
    # Dit zou 'taskId' of een vergelijkbare kolom moeten zijn
    print("Kolommen in fixed_tasks_normalized:")
    print(fixed_tasks_normalized.columns.tolist())
    print("\nKolommen in tasks_normalized:")
    print(tasks_normalized.columns.tolist())
    
    # Bepaal de juiste kolom naam voor task_id in fixed_tasks_normalized
    # Dit kan 'taskId', 'task_id', of een andere variant zijn
    task_id_col = None
    for col in fixed_tasks_normalized.columns:
        if 'task' in col.lower() and 'id' in col.lower():
            task_id_col = col
            break
    
    if task_id_col:
        # Merge om location_id toe te voegen
        # Match op zowel de task_id als de route id
        fixed_tasks_normalized = fixed_tasks_normalized.merge(
            tasks_normalized[['task_id', 'id', 'location_id']],
            left_on=[task_id_col, 'id'],
            right_on=['task_id', 'id'],
            how='left',
            suffixes=('', '_from_tasks')
        )
        
        # Verwijder de extra task_id kolom als die is aangemaakt
        if 'task_id' in fixed_tasks_normalized.columns and task_id_col != 'task_id':
            fixed_tasks_normalized = fixed_tasks_normalized.drop(columns=['task_id'])
        
        print(f"\nlocation_id toegevoegd aan fixed_tasks_normalized")
        print(f"Aantal fixed tasks met location_id: {fixed_tasks_normalized['location_id'].notna().sum()}")
        print(f"Aantal fixed tasks zonder location_id: {fixed_tasks_normalized['location_id'].isna().sum()}")
        
        display(fixed_tasks_normalized.head(10))
    else:
        print("Kan geen task_id kolom vinden in fixed_tasks_normalized")
        display(fixed_tasks_normalized.head())
else:
    print("fixed_tasks_normalized is leeg")

Kolommen in fixed_tasks_normalized:
['taskId', 'activityType', 'fixedPosition', 'id']

Kolommen in tasks_normalized:
['task_id', 'latitude', 'longitude', 'from', 'till', 'id', 'location_id']

location_id toegevoegd aan fixed_tasks_normalized
Aantal fixed tasks met location_id: 1385902
Aantal fixed tasks zonder location_id: 0


,taskId,activityType,fixedPosition,id,location_id
0,64869,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,3
1,64947,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,4
2,64971,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,5
3,64878,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,5
4,64926,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,6
5,64933,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,7
6,64856,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,8
7,64891,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,9
8,64866,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,10
9,64872,Task,True,cb9638db-2817-4ccf-b226-7775c01793a8,11


In [ ]:
display(ingelezen_dataframe.head())
display(tasks_normalized.head())
display(fixed_tasks_normalized.head())